In [ ]:
import numpy as np
import pandas as pd
import warnings
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
warnings.filterwarnings('ignore')
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize =(12,8))

In [ ]:
df = pd.read_csv('/content/TWO_CENTURIES_OF_UM_RACES.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.info

In [ ]:
df.drop_duplicates(inplace= True)

In [ ]:
df

# **How many events in each country?**

In [ ]:
df['Event name'].str.split('(').str.get(1).str.split(')').str.get(0).value_counts().head(10).plot(kind='bar',title='No of Events in each country',ylabel='Count',xlabel='Country',color='lightblue',figsize=(10,5))

data transformation (select only france and prepare for analysis)

In [ ]:
df=df[df['Event name'].str.split('(').str.get(1).str.split(')').str.get(0)=='FRA']
df.shape

In [ ]:
df=df[df['Athlete country']=='FRA']
df

In [ ]:
df['Event distance/length'].nunique()

In [ ]:
df['Event distance/length'].value_counts()

keep only most occurs (100km,50km,45km,55km,80km)

In [ ]:
df = df[df['Event distance/length'].isin(['100km','50km','45km','55km','80km'])]
df

Bar graph showing the 10 most frequent event distances/lengths with the number of occurrences for each distance/length.

In [ ]:
df['Event distance/length'].value_counts().head(10).plot(kind='bar',title='no of distace',xlabel='Event distance/length',ylabel='amount of km',color='purple')

In [ ]:
df['Athlete club'].value_counts()

In [ ]:
df['Athlete club'].isnull().sum()

In [ ]:
df['Athlete club'].fillna(0)

In [ ]:
df['Event name'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna(subset=['Athlete year of birth'])
df

In [ ]:
df.isnull().sum()

In [ ]:
df['Athlete year of birth']=df['Athlete year of birth'].astype('int64')
df

In [ ]:
df['Athlete_Age']=df['Year of event']-df['Athlete year of birth']
df

In [ ]:
df['Athlete performance'] = df['Athlete performance'].str.split(" ").str.get(0)
df

In [ ]:
df=df.drop_duplicates()

In [ ]:
df=df.reset_index()

In [ ]:
df['Athlete average speed']=df['Athlete average speed'].astype('float64')

In [ ]:
df = df[df['Athlete average speed'] < 25]
df

In [ ]:
df=df.drop(['Athlete club','Athlete year of birth'],axis=1)

In [ ]:
df.describe().T

In [ ]:
df

In [ ]:
# number of alhlete participated in each event
df['Event name'].value_counts().head().plot(kind='bar',title='Number of Athlete',xlabel='Event Name',ylabel='NO. of Athlete ',
                                           figsize=(10,6),color='red')

The Number of Race in 50 km,100 km,24h and 12h category By gender

In [ ]:
#Athelete participated gender wise in each Event distance/length.
sns.displot(data=df,kind='hist',x='Event distance/length',hue='Athlete gender',binwidth=5)
plt.title('The Number of Race in 100km,50km,45km,55km,80km category By gender')

In [ ]:
race_by_gender=pd.crosstab(index=df['Event distance/length'],columns=df['Athlete gender'])
race_by_gender

Distributions of Athlete age

In [ ]:
sns.displot(data=df,x='Athlete_Age',kind='hist',color='blue',kde='True')
plt.title("Distributions of Athlete age")

Distributions of Athlete average speed

In [ ]:
sns.displot(data=df,x='Athlete average speed',color='grey',kde=True)
plt.title("Distributions of Athlete average speed")

In [ ]:
age_vs_speed=df.groupby(['Athlete age category'])['Athlete average speed'].agg(['mean','count']).sort_values(by='mean',ascending=False).query('count>20')
age_vs_speed

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(age_vs_speed.index, age_vs_speed['mean'], marker='x', color='r', linestyle='--')
plt.xlabel('Age group')
plt.ylabel('Average Speed')
plt.title('Average speed vs Age')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
distance_gender=df.groupby(['Event distance/length','Athlete gender'])['Athlete average speed'].agg(['mean','count']).sort_values(by='mean',ascending=False)
data=distance_gender.reset_index()
data

Athlete average speed by Event Distance and Gender

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='Event distance/length', y='mean', hue='Athlete gender', data=data)
plt.xlabel('Event distance/length')
plt.ylabel('Athlete average speed')
plt.title('Athlete average speed by Event Distance and Gender')
plt.legend(title='Athlete Gender')
plt.show()

In [ ]:
distance = df.groupby(['Event distance/length','Athlete gender']).agg({'Athlete gender': 'count', 'Athlete average speed': ['mean', 'count']})
distance

the relationship between the length of the race and athlethe average speed

In [ ]:
# the relationship between the length of the race and athlethe average speed
plt.figure(figsize=(12, 6))
sns.violinplot(data =df, x = 'Event distance/length', y='Athlete average speed', hue = 'Athlete gender', split = True, inner='quartz')
plt.tight_layout()

In [ ]:
#this will covert event date to fix format will will be easily Accessible
df['Event dates']=pd.to_datetime(df['Event dates'],errors='coerce')

In [ ]:
df['Athlete performance']=pd.to_datetime(df['Athlete performance'],errors='coerce')

In [ ]:
filtered_df = df[(df['Event distance/length'] == '80km') | (df['Event distance/length'] == '100km')]
sns.boxplot(y=filtered_df['Athlete performance'].dt.hour,x=filtered_df['Event distance/length'])

In [ ]:
filtered_df

Time in hours vs Speed

In [ ]:
plt.figure(figsize=(16,8))
# Pass the entire filtered_df DataFrame as the data argument
sns.pointplot(data=filtered_df, x=filtered_df['Athlete performance'].dt.hour, y=df['Athlete average speed'],estimator='mean')
plt.title('Time vs Speed')

In [ ]:
df['Athlete_performance_in_hours']=df['Athlete performance'].dt.hour+(df['Athlete performance'].dt.minute/60)

In [ ]:
events_by_hours=df.groupby('Event name')['Athlete_performance_in_hours'].agg(['mean','count','min','max']).query('count>20')
events_by_hours

In [ ]:
most_performance_time=events_by_hours.sort_values(by='mean',ascending=False)
most_performance_time

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(most_performance_time.head().index, most_performance_time.head()['mean'], marker='o', color='b', linestyle='-')
plt.xlabel('Event name')
plt.ylabel('Time in hours')
plt.title('Top 5 Event name that has most performance time in hours')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
fastest_Finish=events_by_hours.sort_values(by='min',ascending=True)
fastest_Finish

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(fastest_Finish.head().index, fastest_Finish.head()['min'])
plt.xlabel('Event name')
plt.ylabel('Time in hours')
plt.title('Top 5 Event with fastest Finish')
plt.xticks(rotation=45)
plt.show()

In [ ]:
Slowest_Finish=events_by_hours.sort_values(by='max',ascending=False)
Slowest_Finish

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(Slowest_Finish.head().index, Slowest_Finish.head()['max'],color='green')
plt.xlabel('Event name')
plt.ylabel('Time in hours')
plt.title('Top 5 Event with slowest Finish')
plt.xticks(rotation=45)
plt.show()

In [ ]:
df['seasons']=df['Event dates'].dt.month.apply(lambda x: 'winter' if x > 11 else 'fall' if x > 8 else 'summer' if x > 3 else 'spring' if x > 2 else 'winter')
df

In [ ]:
season_count=df.seasons.value_counts()
season_count

In [ ]:
px.pie(names=season_count.index,values=season_count.values,hole=.6)

In [ ]:
#comparing the average speed by season and gender
Season_race = df.groupby(['seasons','Athlete gender']).agg({'Athlete gender': 'count', 'Athlete average speed': ['mean', 'count']})
Season_race

In [ ]:
Season_race = df.groupby('seasons')['Athlete average speed'].agg(['mean', 'count']).sort_values(by='mean',ascending=False)
Season_race

In [ ]:
sns.relplot(data = Season_race , x = 'seasons' , y = 'mean',kind = 'scatter' )
plt.title('comparing the average speed by season')
plt.show()